In [1]:
from enum import Enum
from queue import LifoQueue
from random import random
from typing import TypeVar, Callable, Sequence, List
from bisect import bisect
from spacy.language import Language
from spacy.tokenizer import Tokenizer
from spacy.vocab import Vocab

X = TypeVar('X')

# edit distance

In [2]:
class Operation(Enum):
    DELETE = 0
    CHANGE = 1
    INSERT = 2
    NO_CHANGE = 3


class EditTableCell:

    def __init__(self, weight: int, parent: 'EditTableCell' = None) -> None:
        super().__init__()
        self.weight = weight
        self.parent = parent
        self.comparator_weight = weight
        self.operation: Operation = Operation.NO_CHANGE

    def edit_sequence(self, queue = None):
        if queue is None:
            queue = LifoQueue()
        queue.put(self.operation)
        if self.parent is not None:
            return self.parent.edit_sequence(queue)
        return queue

def edit_distance(x: Sequence[X], y: Sequence[X], delta: Callable[[X, X], int] = lambda a, b: 0 if a == b else 1):
    edit_table = [[EditTableCell(0) for _ in range(len(y) + 1)] for _ in range(len(x) + 1)]
    for i in range(1, len(x) + 1):
        edit_table[i][0].parent = edit_table[i - 1][0]
        edit_table[i][0].weight = i
        edit_table[i][0].operation = Operation.DELETE
    for i in range(1, len(y) + 1):
        edit_table[0][i].parent = edit_table[0][i - 1]
        edit_table[0][i].weight = i
        edit_table[0][i].operation = Operation.INSERT
    for i in range(len(x)):
        k = i + 1
        for j in range(len(y)):
            l = j + 1
            edit_table[k - 1][l].comparator_weight = edit_table[k - 1][l].weight + 1
            edit_table[k][l - 1].comparator_weight = edit_table[k][l - 1].weight + 1
            edit_table[k - 1][l - 1].comparator_weight = edit_table[k - 1][l - 1].weight + delta(x[i], y[j])
            parent = min([edit_table[k - 1][l],
                          edit_table[k][l - 1],
                          edit_table[k - 1][l - 1]], key=lambda a: a.comparator_weight)
            if edit_table[k - 1][l] == parent:
                edit_table[k][l].operation = Operation.DELETE
            elif edit_table[k][l - 1] == parent:
                edit_table[k][l].operation = Operation.INSERT
            elif edit_table[k - 1][l - 1] == parent:
                if delta(x[i], y[j]) == 0:
                    edit_table[k][l].operation = Operation.NO_CHANGE
                else:
                    edit_table[k][l].operation = Operation.CHANGE
            edit_table[k][l].weight = parent.comparator_weight
            edit_table[k][l].parent = parent
    q = edit_table[len(x)][len(y)].edit_sequence()
    result = x
    print(result)
    x_i = 0
    y_i = 0
    q.get()
    while not q.empty():
        operation = q.get()
        if operation == Operation.INSERT:
            result = result[:x_i] + y[y_i] + result[x_i:]
            y_i += 1
            x_i += 1
            print(result)
        elif operation == Operation.DELETE:
            result = result[:x_i] + result[x_i + 1:]
            print(result)
        elif operation == operation.CHANGE:
            result = result[:x_i] + y[y_i] + result[x_i + 1:]
            y_i += 1
            x_i += 1
            print(result)
        elif operation == operation.NO_CHANGE:
            y_i += 1
            x_i += 1
    return edit_table[len(x)][len(y)].weight

In [3]:
edit_distance('los', 'kloc')

los
klos
kloc


2

In [4]:
edit_distance('Łódź', 'Lodz')

Łódź
Lódź
Lodź
Lodz


3

In [5]:
edit_distance('kwintesencja', 'quintessence')

kwintesencja
qwintesencja
quintesencja
quintessencja
quintessencea
quintessence


5

In [6]:
edit_distance('ATGAATCTTACCGCCTCG', 'ATGAGGCTCTGGCCCCTG')

ATGAATCTTACCGCCTCG
ATGAGTCTTACCGCCTCG
ATGAGGCTTACCGCCTCG
ATGAGGCTCTACCGCCTCG
ATGAGGCTCTGCCGCCTCG
ATGAGGCTCTGGCCGCCTCG
ATGAGGCTCTGGCCCCTCG
ATGAGGCTCTGGCCCCTG


7

# longest common subsequence

In [7]:
def lcs(x: Sequence[X], y: Sequence[X]):
    ranges = [len(y)]
    y_letters = list(y)
    for i in range(len(x)):
        positions = [j for j, l in enumerate(y_letters) if l == x[i]]
        positions.reverse()
        for p in positions:
            k = bisect(ranges, p)
            if k == bisect(ranges, p - 1):
                if k < len(ranges) - 1:
                    ranges[k] = p
                else:
                    ranges[k:k] = [p]
    return len(ranges) - 1

In [8]:
def remove_tokens(tokens: List):
    res = []
    for t in tokens:
        if random() >= 0.03:
            res.append(t)
    return res

In [18]:
with open('romeo-i-julia.txt', "r") as file:
    text = file.read()
    vocab = Language(Vocab()).vocab
    tokenizer = Tokenizer(vocab)
    tokens = tokenizer(text)
    text1 = remove_tokens(tokens)
    text2 = remove_tokens(tokens)
    with open('text1.txt', 'w') as new_file:
        for token in text1:
            new_file.write(token.text_with_ws)
    with open('text2.txt', 'w') as new_file:
        for token in text2:
            new_file.write(token.text_with_ws)
    print(len(tokens))
    print(len(text1))
    print(len(text2))
    print(lcs(text1, text2))

26689
25870
25916
25127


# diff

In [19]:
def diff(a: Sequence[X], b: Sequence[X]):
    L = [[0 for _ in range(len(b) + 1)] for _ in range(len(a) + 1)]
    for i in range(len(a) + 1):
        for j in range(len(b) + 1):
            if i == 0 or j == 0:
                L[i][j] = 0
            elif a[i - 1] == b[j - 1]:
                L[i][j] = L[i - 1][j - 1] + 1
            else:
                L[i][j] = max(L[i - 1][j], L[i][j - 1])
    lines = []
    i = len(a) - 1
    j = len(b) - 1
    while i >= 0 and j >= 0:
        if a[i] == b[j]:
            i, j = i-1, j-1
        elif L[i][j-1] >= L[i-1][j]:
            lines.append(f"> [{j}] {b[j]}")
            j -= 1
        elif L[i][j-1] < L[i-1][j]:
            lines.append(f"< [{i}] {a[i]}")
            i -= 1
    while j >= 0:
        lines.append(f"> [{j}] {b[j]}")
        j -= 1
    while i >= 0:
        lines.append(f"< [{i}] {a[i]}")
        i -= 1
    lines.reverse()
    for line in lines:
        print(line)
    return L

< [12]  * MONTEKI, KAPULET — naczelnicy dwóch domów sobie

> [12]  * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie

< [13]  * STARZEC — stryjeczny brat Kapuleta

< [14]  * — syn Montekiego

> [13]  * STARZEC — stryjeczny brat Kapuleta

> [14]  * ROMEO — syn Montekiego

< [18]  * LAURENTY — franciszkanin

> [18]  * LAURENTY — ojciec franciszkanin

< [19]  * JAN — brat z tegoż zgromadzenia

< [20]  * BALTAZAR — służący Romea* SAMSON, GRZEGORZ — słudzy Kapuleta

> [19]  * JAN — brat z tegoż zgromadzenia

> [20]  * BALTAZAR — służący Romea

> [21]  * SAMSON, GRZEGORZ — słudzy Kapuleta

< [27]  * PANI MONTEKI — małżonka Montekiego

> [28]  * PANI — małżonka Montekiego

< [36] Rzecz odbywa się przez większą część sztuki w część piątego aktu w Mantui.

> [37] Rzecz odbywa się przez większą część sztuki w Weronie, przez część piątego aktu w Mantui.

< [42] Przełożył Jan Kasprowicz

> [43] Przełożył KasprowiczDwa rody, zacne jednako i sławne —

< [43] 

< [44] Dwa rody, zacne j

> [5035] Cóż jest niegodniej, cóż jest większym grzechem:

< [5054] Czy tak mię kusić do krzywoprzysięstwa?

< [5055] Czy lżyć małżonka mego tymiż usty,

> [5036] Czy tak mię kusić do krzywoprzysięstwa?

> [5037] Czy lżyć małżonka mego tymiż usty,Którymi tyle razy go pod nieboWznosiłaś chwaląc? Precz, uwodzicielko!

< [5056] Którymi tyle razy go pod niebo

< [5057] Wznosiłaś chwaląc? Precz, uwodzicielko!

< [5080] W ten zatem? To bardzo pośpiesznie.

> [5060] W ten czwartek zatem? To bardzo pośpiesznie.

< [5085] Mój teść, Kapulet, życzy sobie 

> [5065] Mój teść, Kapulet, życzy sobie tego;

< [5089] OJCIEC LAURENTY

> [5069] OJCIEC LAURENTYNie znasz pan, mówisz, uczuć swojej przyszłej;

< [5090] 

< [5091] Nie znasz pan, mówisz, uczuć swojej przyszłej;

< [5098] Małom jej przeto mówił o miłości,

> [5076] Małom jej przeto miłości,

< [5099] Bo w domu łez nie uśmiecha.

< [5100] Ojciec jej, mając to za niebezpieczne,

> [5077] Bo Wenus domu łez się nie uśmiecha.

> [5078] Ojciec mając 